# Exercise 9
FIRSTNAME LASTNAME
## Task 2 - Rectification

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import h5py

In [ ]:
# load data
with h5py.File('task2.mat', 'r') as file:
    data = {key: np.array(file[key]).T for key in file.keys()}

F = data["F"]
points_l = data["points_l"]
points_r = data["points_r"]
Ml = data["Ml"]
Mr = data["Mr"]

In [ ]:
imgl = cv.imread("books1.jpg", cv.IMREAD_GRAYSCALE)
imgr = cv.imread("books2.jpg", cv.IMREAD_GRAYSCALE)

In [ ]:
# TODO compute epipoles once more
el = ...
er = ...

In [ ]:
def deshear(img, H):
    # this is already fully implemented and used, you do not need to do anything with this function
    h, w = img.shape
    hh, hw = np.array(img.shape) // 2
    a = H @ np.array([hw, 1, 1])
    a /= a[-1]
    b = H @ np.array([w, hh, 1])
    b /= b[-1]
    c = H @ np.array([hw, h, 1])
    c /= c[-1]
    d = H @ np.array([1, hh, 1])
    d /= d[-1]
    x = b - d
    y = c - a
    k1 = (h**2 * x[1] ** 2 + w**2 * y[1] ** 2) / (h * w * (x[1] * y[0] - x[0] * y[1]))
    k2 = (h**2 * x[0] * x[1] + w**2 * y[0] * y[1]) / (h * w * (x[0] * y[1] - x[1] * y[0]))

    Hn = np.eye(3)
    Hn[0, :2] = [-k1, -k2]
    return Hn


In [ ]:
center = np.array(imgl.shape) // 2


def rectify_right(er, center):
    # TODO implement rectify right
    
    # TODO: Define matrix G
    
    # TODO: Define transformation matrix T 
    # Moves image center to (0,0,1)
    
    # TODO: Define transformation matrix R
    # Moves translated epipole (e1',e2',1) into (f,0,1)
    
    Hr = ...


Hr = rectify_right(er, center)
Hrf = deshear(imgr, Hr) @ Hr


In [ ]:
def rectify_left(pl, pr, Mr, Hr):
    # TODO implement rectify left
    
    # TODO: Check that fundamental matrix F is of form [e_r]_x M
    # Hint: What needs to hold for this to be true?
    ...


Hl = rectify_left(points_l, points_r, Mr, Hrf)
Hlf = deshear(imgl, Hl) @ Hl


## Helper functions

In [ ]:
import numpy as np
from scipy.interpolate import LinearNDInterpolator
from skimage import img_as_float


def interpolate_transform_image(I, H):
    # Create meshgrid over the image
    xl, yl = np.meshgrid(
        np.arange(I.shape[1]), 
        np.arange(I.shape[0]), 
        indexing="xy"
    )
    # Turn the 2D grid into 1D arrays
    p1 = np.vstack((xl.ravel(), yl.ravel(), np.ones(xl.size)))

    # Transform the points and normalize
    p2 = H @ p1
    p2 /= p2[2]

    # Interpolate the image to the points
    I = img_as_float(I)
    cL = I.ravel()
    F = LinearNDInterpolator(p2[:2].T, cL)

    return F, p2


def warp_images(Fl, pl2, Fr, pr2):
    # Find min and max values
    minL = np.min(pl2[:2, :], axis=1)
    maxL = np.max(pl2[:2, :], axis=1)
    minR = np.min(pr2[:2, :], axis=1)
    maxR = np.max(pr2[:2, :], axis=1)

    minLR = np.minimum(minL, minR)
    maxLR = np.maximum(maxL, maxR)

    # Create meshgrid over the min and max values
    X, Y = np.meshgrid(
        np.arange(np.floor(minLR[0]), np.ceil(maxLR[0])+1),
        np.arange(np.floor(minLR[1]), np.ceil(maxLR[1])+1),
        indexing='xy'
    )
    Il1 = Fl(X, Y)
    Ir1 = Fr(X, Y)

    return Il1, Ir1, minLR


## Visualize the results

In [ ]:
# interpolate and show stuff
Fr, pr2 = interpolate_transform_image(imgr, Hrf)
Fl, pl2 = interpolate_transform_image(imgl, Hlf)

wil, wir, minlr = warp_images(Fl, pl2, Fr, pr2)

pln = Hlf @ points_l
pln = pln[:2] / pln[2] - minlr[:, None]
prn = Hrf @ points_r
prn = prn[:2] / prn[2] - minlr[:, None]

h, w = wil.shape
for ptl, ptr in zip(pln.T, prn.T):
    plt.plot([0, w*2], [ptl[1], ptr[1]], "-c", lw=1, ms=0)
    plt.plot([ptl[0], ptr[0] + w], [ptl[1], ptr[1]], "or", lw=1, ms=3)


plt.imshow(np.concatenate([wil, wir], axis=1), cmap="gray")
plt.tight_layout()
plt.axis("off")
plt.show()

## Questions and answers: